# LangChain: Agents

Agents in LangChain are designed to use a language model to determine a sequence of actions to take. Unlike chains, where the sequence of actions is hardcoded, agents use a language model as a reasoning engine to decide which actions to take and in what order. Agents are composed of several key components.

**Agent:** This is the chain responsible for deciding the next step to take. It takes inputs such as available tools, user input, and any previously executed steps. The agent then returns either the next action to take or the final response to send to the user.

**Tools:** Tools are functions that an agent calls to perform specific tasks. They are essential for the agent to accomplish its objective. LangChain provides a wide range of tools, and custom tools can also be defined.

**Toolkits:** Toolkits are groups of tools needed to achieve specific objectives. They consist of 3-5 tools and are crucial for an agent to have access to the right set of tools.

**AgentExecutor:** The agent executor is the runtime for an agent. It calls the agent and executes the actions it chooses. It handles complexities such as handling non-existent tools, tool errors, and logging.

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools
* Pandas-df Agent

https://python.langchain.com/docs/integrations/toolkits

In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  # Install the package
  !pip install langchain openai litellm wikipedia -qU
else:
  print('Not running on CoLab')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00


#OpenAI API

Let us try first using Openai API directly

Read about the OpenAI API specifications and endpoints on below link

https://platform.openai.com/docs/api-reference

To learn about the available Models

https://platform.openai.com/docs/models/gpt-3-5

In [ ]:
import os
import openai

# select the model to use
llm_model = "gpt-3.5-turbo-0613"

# openai.api_key = ""

os.environ['OPENAI_API_KEY'] = ""

# os.environ["OPENAI_API_KEY"] = "sk-litellm-7_NPZhMGxY2GoHC59LgbDw" # [OPTIONAL] replace with your openai key

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

## Built-in LangChain tools

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
# define the external tools used to perform the task
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [ ]:
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
I can use the calculator tool to find the answer to this question.

Action:
```json
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```
Observation: Answer: 75.0
Thought:The answer is 75.0.
Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

## Wikipedia example

In [ ]:
question = "Elon Musk is an American / African entrepreneur \
and the Founder of Many companies\
what the summaries of companies He founded?"
result = agent(question)



> Entering new AgentExecutor chain...
Question: What are the summaries of the companies Elon Musk founded?
Thought: I can use Wikipedia to find information about the companies Elon Musk founded.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "List of companies founded by Elon Musk"
}
```

Observation: Page: Acquisition of Twitter by Elon Musk
Summary: Business magnate Elon Musk initiated an acquisition of American social media company Twitter, Inc. on April 14, 2022, and concluded it on October 27, 2022. Musk had begun buying shares of the company in January 2022, becoming its largest shareholder by April with a 9.1 percent ownership stake. Twitter invited Musk to join its board of directors, an offer he initially accepted before declining. On April 14, Musk made an unsolicited offer to purchase the company, to which Twitter's board responded with a "poison pill" strategy to resist a hostile takeover before unanimously accepting Musk's buyout offer of $44 billion on April 2

InvalidRequestError: ignored

## Python Agent

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Amjad", "Raza"],
                 ["Irfan", "Aslam"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")



> Entering new AgentExecutor chain...


To sort the customers by last name and then first name, I can use the `sorted()` function and provide a custom key function that returns a tuple of the last name and first name. This will ensure that the sorting is done in the desired order.
Action: Python_REPL
Action Input: sorted([['Amjad', 'Raza'], ['Irfan', 'Aslam'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: 
Thought:The code executed successfully and returned the sorted list of customers by last name and then first name.
Final Answer: [['Irfan', 'Aslam'], ['Jen', 'Ayai'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Amjad', 'Raza'], ['Dolly', 'Too']]

> Finished chain.


"[['Irfan', 'Aslam'], ['Jen', 'Ayai'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Amjad', 'Raza'], ['Dolly', 'Too']]"

#### View detailed outputs of the chains

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Amjad', 'Raza'], ['Irfan', 'Aslam'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Amjad', 'Raza'], ['Irfan', 'Aslam'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you 

## Pandas DataFrame Agent
In this section, we explore, how can we chat with pandas DataFrame using create_pandas_dataframe_agent provided with langchain API. Once, code is ready, we can go and integrate the code in the App.

In [ ]:

from langchain.agents import AgentType
from langchain.agents import create_pandas_dataframe_agent
from langchain.callbacks.streamlit import StreamlitCallbackHandler
from langchain.chat_models import ChatOpenAI

In [ ]:
#Loading CSV file
import pandas as pd
file_path ='https://raw.githubusercontent.com/gventuri/pandas-ai/main/examples/data/Loan%20payments%20data.csv'
df = pd.read_csv(file_path)

In [ ]:
df.head()

,Loan_ID,loan_status,Principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,Gender
0,xqd20166231,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/14/2016 19:31,NaN,45,High School or Below,male
1,xqd20168902,PAIDOFF,1000,30,9/8/2016,10/7/2016,10/7/2016 9:00,NaN,50,Bechalor,female
2,xqd20160003,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/25/2016 16:58,NaN,33,Bechalor,female
3,xqd20160004,PAIDOFF,1000,15,9/8/2016,9/22/2016,9/22/2016 20:00,NaN,27,college,male
4,xqd20160005,PAIDOFF,1000,30,9/9/2016,10/8/2016,9/23/2016 21:36,NaN,28,college,female


In [ ]:

llm = ChatOpenAI(
            temperature=0, model="gpt-3.5-turbo-0613",streaming=True
        )

pandas_df_agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
)


In [ ]:
# Enter Prompt related to data or Select from Pre-defined for demo purposes.

prompt = 'How many loans are from Women that have been paid off?' #@param [ "How many loans are from Women that have been paid off?",

In [ ]:
langchain.debug=True
response = pandas_df_agent.run(prompt)
langchain.debug=True

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How many loans are from Women that have been paid off?"
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nThis is the result of `print(df.head())`:\n|    | Loan_ID     | loan_status   |   Principal |   terms | effective_date   | due_date   | paid_off_time   |   past_due_days |   age | education            | Gender   |\n|---:|:------------|:--------------|------------:|--------:|:-----------------|:-----------|:----------------|----------------:|------:|:---------------------|:---------|\n|  0 | xqd20166231 | PAIDOFF       |        1000 |      30 | 9/8/2016         | 10/7/2016  | 9/14/2016 19:31 |             nan |    45 | High School or Below | male     |\n|  1 | xqd20168902 | PAIDOFF       |        1000 |      30 | 9/8/2016         | 10/7/2016  | 10/7

**Note**:

The agent will sometimes come to the wrong conclusion (agents are a work in progress!).

If it does, please try running it again.

In [ ]:
try:
    result = agent("whats the date today?")
except:
    print("exception on external access")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "whats the date today?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "whats the date today?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return \"I don't know\" as the answer.\n\n\nPython_REPL: A Python shell. Use this to execute pyt

# Conclusions
The implementation of agents within the LangChain ecosystem represents a significant step forward in the realm of decentralized and autonomous systems. However, there are some risks involved and users/developers  have to be extra cautious while using them.

1. The LLM costs can be huge due to steps
2. The executors can run anything and potentially harm your system
3. The number of steps & output is not predicted.

